In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from string import ascii_lowercase
from PIL import Image
from pprint import pprint
from tqdm import tqdm

In [51]:
files = []
for s in ascii_lowercase[:6]:
    files.append("{}.txt".format(s))
print(files)

['a.txt', 'b.txt', 'c.txt', 'd.txt', 'e.txt', 'f.txt']


In [52]:
# Read the input
# office has -1 for #
#            -2 for devs
#            -3 for managers
EMPTY = -1
DEV = -2
MANAGER = -3 
def read_input(filename):
    with open(filename, 'r') as f:
        w, h = [int(x) for x in f.readline().split()]
        office_c = []
        for i in range(h):
            office_c.append(f.readline())
        office = []
        for i in range(len(office_c)):
            office.append([])
            for j in range(len(office_c[i])):
                if office_c[i][j] == '#':
                    office[i].append(-1)
                if office_c[i][j] == '_':
                    office[i].append(-2)
                if office_c[i][j] == 'M':
                    office[i].append(-3)
        d = int(f.readline())
        devs = []
        for i in range(d):
            line = f.readline()
            devs.append({
                "company": line.split()[0],
                "bonus": int(line.split()[1]),
                "skills": set([x for x in line.split()[3:]]),
                "id": i
                })
        pm = int(f.readline())
        managers = []
        for i in range(pm):
            line = f.readline()
            managers.append({
                "company": line.split()[0],
                "bonus": int(line.split()[1]),
                "id": i
                })
        return office, devs, managers

In [63]:
filename = files[1]
office, devs, managers = read_input(filename)
df_devs = pd.DataFrame(devs)
df_devs['x'] = np.array([-1 for i in range(len(df_devs))])
df_devs['y'] = np.array([-1 for i in range(len(df_devs))])
df_managers = pd.DataFrame(managers)
df_managers['x'] = np.array([-1 for i in range(len(df_managers))])
df_managers['y'] = np.array([-1 for i in range(len(df_managers))])


In [64]:
with open(filename.split(".")[0]+".out", 'r') as f:
    for i, d in df_devs.iterrows():
        l = f.readline()
        if "X" in l:
            pass
        else:
            df_devs.loc[i, 'x'] = int(l.split()[0])
            df_devs.loc[i, 'y'] = int(l.split()[1])
    for i, m in df_managers.iterrows():
        l = f.readline()
        if "X" in l:
            pass
        else:
            df_managers.loc[i, 'x'] = int(l.split()[0])
            df_managers.loc[i, 'y'] = int(l.split()[1])
df_devs

,company,bonus,skills,id,x,y
0,ccf1bb,9,"{sa8eag, suiohy, sc3ltx, s47grn, siktgp, stbl1...",0,42,56
1,cv3f6j,9,"{s3vpte, sprncf, sd8ae2, scq7ky, sbfpoi, s7t7n...",1,14,85
2,c5lvo1,8,"{siktgp, skbwiw, s18745, sd61kw, spgxtp, sye8x...",2,94,76
3,c2rf54,9,"{shhbpd, sc3ltx, sd61kw, s8if57, s2eqln, s0azd...",3,37,83
4,c7vpb1,13,"{siktgp, suiohy, sd61kw, sc3ltx, s2eqln, sk2gg...",4,40,11
...,...,...,...,...,...,...
6831,c7lfl5,14,"{siktgp, skbwiw, shhbpd, sf1vti, sd61kw, spgxt...",6831,76,30
6832,cnb5vt,13,"{sr68ft, sgwweq, savkaj, szrdiw, ser4bt, sbzio...",6832,68,39
6833,cy8hnj,13,"{sr3t3a, s3vpte, sgwweq, sl4ql5, sprncf, sd8ae...",6833,84,17
6834,c2rf54,12,"{siktgp, sa8eag, s18745, sf1vti, stbl1i, s0azd...",6834,30,42


In [65]:
for i, dev in df_devs.iterrows():
    if dev['x'] != -1:
        office[dev['y']][dev['x']] = dev['id']
for i, man in df_managers.iterrows():
    if man['x'] != -1:
        office[man['y']][man['x']] = man['id'] + len(df_devs)


In [66]:
for i in range(len(office)):
    for j in range(len(office[i])):
        if office[i][j] == MANAGER:
            # Put a random manager.
            companies = set()
            for k in range(-1,2):
                for l in range(-1,2):
                    if len(office )> i+k >=0 and len(office[i]) > j+l >= 0:
                        if len(df_devs) > office[i+k][j+l] >=0:
                            companies.add(df_devs.iloc[office[i+k][j+l]]['company'])
                        elif office[i+k][j+l] >= len(df_devs):
                            companies.add(df_managers.loc[office[i+k][j+l] - len(df_devs)]['company'])
            if len(df_managers[df_managers['company'].isin(companies)]) > 0:
                best = df_managers[df_managers['company'].isin(companies)]
            elif len(df_managers[df_managers['x']==-1]) > 0:
                man = df_managers[df_managers['x']==-1].sample(1).iloc[0]
                df_managers.loc[man['id'], 'x'] = j
                df_managers.loc[man['id'], 'y'] = i
                office[i][j] = df_managers.loc[man['id'], 'id'] + len(df_devs)

In [67]:
def print_output(df_devs, df_managers, outfile):
    with open(outfile, 'w') as f:
        for i, l in df_devs.iterrows():
            if l.x == -1:
                print("X", file=f)
            else:
                print("{} {}".format(l.x, l.y), file=f)
        for i, l in df_managers.iterrows():
            if l.x == -1:
                print("X", file=f)
            else:
                print("{} {}".format(l.x, l.y), file=f)

In [68]:
print_output(df_devs, df_managers, filename.split(".")[0] + ".out")

# Optimize results

In [69]:
def compute_points(dev_1, empl2):
    if empl2 >= len(df_devs):
        man_2 = df_managers.loc[empl2-len(df_devs)]
        if man_2['company'] == dev_1['company']:
            return man_2['bonus'] * dev_1['bonus']
        else:
            return 0
    else:
        dev_2 = df_devs.loc[empl2]
        points = dev_1['skills'].intersect(dev_2['skills']) * (dev_1['skills'].union(dev_2['skills']) - dev_1['skills'].intersect(dev_2['skills']))
        if dev_2['company'] == dev_1['company']:
            points += dev_2['bonus'] * dev_1['bonus']
        return points
        

In [ ]:
# Try only with devs
for i in range(1000):
    improved = False
    while not improved:
        dev_1 = df_devs.sample(1).iloc[0]
        dev_2 = df_devs.sample(1).iloc[0]
        start_points_1 = 0
        points = 0
        for k in range(-1,2):
                for l in range(-1,2):
                    if len(office)> dev_1['y']+k >=0 and len(office[i]) > dev_1['x']+l >= 0:
                        if office[ dev_1['y']+k][dev_1['x']+l] >= 0:
                            start_points_1 += compute_points(dev_1, office[ dev_1['y']+k][dev_1['x']+l])
                    if len(office)> dev_2['y']+k >=0 and len(office[i]) > dev_2['x']+l >= 0:
                        if office[ dev_2['y']+k][dev_2['x']+l] >= 0:
                            start_points_1 += compute_points(dev_2, office[ dev_2['y']+k][dev_2['x']+l])
        tmp = office[dev_1['y']][dev_1['x']]
        office[dev_1['y']][dev_1['x']] = office[dev_2['y']][dev_2['x']]
        office[dev_2['y']][dev_2['x']] = tmp
        
        tmp = dev_1['y']
        dev_1['y'] = dev_2['y']
        dev_2['y'] = tmp
                        
        tmp = dev_1['x']
        dev_1[''] = dev_2['y']
        dev_2['y'] = tmp